In [0]:
data = [(1, "Steven",5000), (2, "Neena",4000), (3, "John",6000), (4, "Peter",7000), (5,"Bob",3000)]
Schema = "Id int, Name string, Salary int"
df = spark.createDataFrame(data, Schema)
df.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
from pyspark.sql.functions import lit, current_date

df_dim = df.withColumn("Flag", lit("Y"))\
    .withColumn("Eff_date", current_date().cast("date"))\
    .withColumn("Exp_date", lit("9999-12-31").cast("date"))
df_dim.display()

In [0]:
data = [(1, "Steven",1000), (3, "Neena",2000), (32, "Joohn",6000), (4, "Petern",7000), (5,"Bob",3000)]
Schema = "Id int, Name string, Salary int"
df_src = spark.createDataFrame(data, Schema)
df_src.display()

In [0]:
dim_current = df_dim.filter(col("Flag") == 'Y')
dim_current.display()

In [0]:
joined =df_src.alias("src").join(df_dim.alias("dim") ,df_src['id'] == df_dim['id'], 'left')
joined.display()

In [0]:
change_cols = ['Name', 'Salary']
with_hash = (
    joined.withColumn("src_hash", sha2(concat_ws("||", *[col(f"src.{c}") for c in change_cols]), 256))
    ).withColumn("dim_hash", sha2(concat_ws("||", *[col(f"dim.{c}") for c in change_cols]), 256))


In [0]:
changes = with_hash.withColumn("Change_Type", 
                               when(col("dim.Id").isNull(), 'INSERT').when(col("src_hash") != col("dim_hash"), "UPDATE")).filter(col("Change_Type").isNotNull())
changes.display()

In [0]:
expired = changes.filter(col("Change_Type") == 'UPDATE').select("dim.Id", "dim.Name", "dim.Salary","dim.Eff_date").withColumn("Exp_date", lit("9999-12-31").cast("date")). withColumn("Flag", lit("N"))

new_versions = changes.select("src.*").withColumn("Eff_date", current_date()).withColumn("Exp_date", lit(None).cast("date")).withColumn("Flag", lit("Y"))

final_updates = expired.unionByName(new_versions)
final_updates.display()

In [0]:
expired.printSchema()


In [0]:
new_versions.printSchema()